# **Video Steganography Encoder / Decoder**


# **Image Steganography Encoder**

Follow the steps below to properly hide a message in your chosen video. You must run each cell to perform the encoding process. In order to run each cell, hover your mouse on a particular cell then click on the play button at the left side of the cell.
<br>
<br>
**Note**: All video files must be in MP4 format.



In [ ]:
# [OPTIONAL] STEP 1: Install the stegano library

# Restart the session after installing the library
!pip install stegano

In [ ]:
# STEP 2: Import all the necessary libraries to perform the video steganography.

from google.colab import files
from stegano import lsb
from os.path import isfile,join
import time
import cv2
import numpy as np
import math
import os
import shutil
from subprocess import call,STDOUT

In [ ]:
# STEP 3: Run this cell to upload your image file.

uploaded = files.upload()

In [ ]:
# STEP 4: Run this cell to enter the message to be hidden and the video file to hide the message in.

input_string = ""

def main():
    global input_string
    input_string = input("Enter the message to be hidden: ")
    f_name=input("Enter the name of the video including its file extension (.mp4): ")
    frame_extraction(f_name)
    call(["ffmpeg", "-i",f_name, "-q:a", "0", "-map", "a", "tmp/audio.mp3", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)

    encode_string(input_string)
    call(["ffmpeg", "-i", "tmp/%d.png" , "-vcodec", "png", "tmp/encoded_video.mov", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)

    call(["ffmpeg", "-i", "tmp/encoded_video.mov", "-i", "tmp/audio.mp3", "-codec", "copy", "encoded_video.mov", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    print("Encoded video saved!")
    clean_tmp()

def split_string(s_str,count=1000):
    per_c=math.ceil(len(s_str)/count)
    c_cout=0
    out_str=''
    split_list=[]
    for s in s_str:
        out_str+=s
        c_cout+=1
        if c_cout == per_c:
            split_list.append(out_str)
            out_str=''
            c_cout=0
    if c_cout!=0:
        split_list.append(out_str)
    return split_list

def frame_extraction(video):
    if not os.path.exists("./tmp"):
        os.makedirs("tmp")
    temp_folder="./tmp"
    print("Converting to individual frames...")

    vidcap = cv2.VideoCapture(video)
    count = 0

    while True:
        success, image = vidcap.read()
        if not success:
            break
        cv2.imwrite(os.path.join(temp_folder, "{:d}.png".format(count)), image)
        count += 1

def encode_string(input_string,root="./tmp/"):
    split_string_list=split_string(input_string)
    for i in range(0,len(split_string_list)):
        f_name="{}{}.png".format(root,i)
        secret_enc=lsb.hide(f_name,split_string_list[i])
        secret_enc.save(f_name)
    print("Hiding message in frames...")

def clean_tmp(path="./tmp"):
    if os.path.exists(path):
        shutil.rmtree(path)

if __name__ == "__main__":
    main()

In [ ]:
# STEP 5: Download the 'encoded_video.mov' file by clicking the three dots beside at the left navigation bar it then click 'Download'.

# **Video Steganography Decoder**

Upload the video file you want to decode by clicking 'Files' in the left navigation bar, then 'Upload to session storage'. In order to run a code snippet to in each cell, hover your mouse on a particular cell then click on the play button at the left side of the cell.

In [ ]:
def decode_string(video):
    frame_extraction(video)
    secret=[]
    root="./tmp/"
    for i in range(len(input_string)):
        f_name="{}{}.png".format(root,i)
        secret_dec= lsb.reveal(f_name)
        if secret_dec is None:
            break
        secret.append(secret_dec)
    print(''.join([i for i in secret]))
    clean_tmp()

# Replace 'INSERT FILE NAME HERE' with the name of the image file you've uploaded including its file extension (.mov).
# Do NOT clear the single quotation marks.
decode_string('INSERT FILE NAME HERE')